# Optimization of Wind/Solar/Battery/Load portfolio

This notebook provides an example of optimizing a wind/solar/battery/load portfolio in the static environment. 

In the static environment the optimization is done in a single run for the whole period. Data series are provided in the form of a Pandas dataframe.

In [11]:
import sys
sys.path.append("./../src/")
from OptiMind import *

In [12]:
import pandas as pd

First we import the data series from the csv file. These consist of the load, the wind and solar power generation, and the spot market prices.

In [13]:
df = pd.read_csv('./data/data_static.csv')
df.loc[:,'timestamps'] = pd.to_datetime(df.loc[:,'timestamps'])

In [14]:
df

,timestamps,load,wind_generation,solar_generation,spot_price
0,2019-05-29 14:00:00,132.654137,206.592623,257.463356,0.3956
1,2019-05-29 15:00:00,154.113323,119.696063,176.601749,0.4443
2,2019-05-29 16:00:00,165.607048,111.521058,84.108178,0.4906
3,2019-05-29 17:00:00,168.367154,54.599250,21.964314,0.5214
4,2019-05-29 18:00:00,188.294382,29.222222,7.228477,0.5262
...,...,...,...,...,...
6931,2020-03-13 09:00:00,330.182226,469.167502,175.291628,0.0846
6932,2020-03-13 10:00:00,337.691553,469.392140,74.792523,0.0847
6933,2020-03-13 11:00:00,327.122966,468.729785,113.845581,0.0801
6934,2020-03-13 12:00:00,325.853585,465.463099,96.488675,0.0787


We build the agent data input structure. First, we create a dataframe which contains 4 columns with the following names:
- "load": power demand in xW
- "generation": aggregated RES (solar, wind) generation in xW
- "price_sell": price for selling power in currency/xWh
- "price_buy": price for buying power in currency/xWh

where x replaces some metric prefix (e.g k for kilo, M for mega)

Dataframe index should consist of the timestamps

In [15]:
timestamps = pd.Series(df["timestamps"], name='timestamps')
load = pd.Series(df["load"], name='load') 
generation = pd.Series(df["wind_generation"]+df["solar_generation"], name='generation') 
price_sell = pd.Series(df["spot_price"] - 0.04, name='price_sell') 
price_buy = pd.Series(df["spot_price"] + 0.04, name='price_buy') 

data_input = pd.concat([timestamps, load, generation, price_sell, price_buy], axis=1).set_index('timestamps')

In [16]:
data_input

,load,generation,price_sell,price_buy
timestamps,,,,
2019-05-29 14:00:00,132.654137,464.055979,0.3556,0.4356
2019-05-29 15:00:00,154.113323,296.297812,0.4043,0.4843
2019-05-29 16:00:00,165.607048,195.629235,0.4506,0.5306
2019-05-29 17:00:00,168.367154,76.563564,0.4814,0.5614
2019-05-29 18:00:00,188.294382,36.450699,0.4862,0.5662
...,...,...,...,...
2020-03-13 09:00:00,330.182226,644.459130,0.0446,0.1246
2020-03-13 10:00:00,337.691553,544.184663,0.0447,0.1247
2020-03-13 11:00:00,327.122966,582.575366,0.0401,0.1201


Then we pass the dataframe to the "Data" structure to create it

In [17]:
data = Data(static_data = data_input)

We build the agent parameter input structure. We provide the following parameters. The parameters that have default values can be skipped.

Battery parameters
- battery_min_level:               battery minimum energy level as percentage of capacity
- battery_capacity:                battery capacity (xWh)
- battery_charge_max:              battery max charging power (xW)
- battery_discharge_max:           battery max discharging power (xW)
- battery_efficiency_charge:       battery efficiency when charging
- battery_efficiency_discharge:    battery efficiency when discharging
- bel_ini_level:                   battery energy level in the beginning of the planning horizon (xWh), (Default = 0.0)
- bel_fin_level:                   battery energy level at the end of the planning horizon (*Wh), (Default = 0.0)

If bel_ini_level or bel_fin_level < battery_min_level, they automatically set to battery_min_level

Grid paramaters
- grid_energy_import_fee:          grid fee for energy consuption (Currency/xWh)
- grid_energy_export_fee:          grid fee for energy production (Currency/xWh)
- grid_power_fee:                  grid contract fee for power (Currency/xW)
- grid_power_fee_penalty:          grid penalty fee for exceeding contract power level (Currency/xW)
- grid_power_contract:             grid contract power level (xW), (if set to zero then power level is optimized)

Other
- period_length:                   period length relative to hour (e.g if 15min periods, then period_length = 0.25), (Default = 1.0)

In [18]:
agent_parameter = agent_example(
    battery_min_level = 0.0,
    battery_capacity = 1040.0,
    battery_charge_max = 470.0,
    battery_discharge_max = 470.0,
    battery_efficiency_charge = 0.9,
    battery_efficiency_discharge = 0.9,
    bel_ini_level = 0.0,
    bel_fin_level = 0.0,
    grid_energy_import_fee = 0.015,
    grid_energy_export_fee = 0.015,
    grid_power_fee = 111.0, 
    grid_power_fee_penalty = 222,
    grid_power_contract = 0.0)

agent = Agents(agent_static = agent_parameter)

We optimize by passing to the static environment function, the data and agent parameter inputs

In [19]:
solution = static(data, agent)

The solution is a Julia structure with the following fields:
- operation_plan: A dataframe with the optimized operation plan
- grid_power_contract: The optimal power level of the grid contract if parameter grid_power_contract was set to 0.0. Else the value of this paramater (xW)
- peak_contract_difference: The difference between power peak and power contract level (xW)
- grid_power_cost: The grid power cost (Currency)   
- total_cost: The value of the objective function that corresponds to the total cost (Currency)
    
Julia DataFrame is converted back to Pandas DataFrame using the toPandasDf function    

In [20]:
operation_plan = toPandasDf(solution.operation_plan).set_index(['timestamps'])
grid_power_contract = solution.grid_power_contract
peak_contract_difference = solution.peak_contract_difference
grid_power_cost = solution.grid_power_cost
total_cost = solution.total_cost

In [21]:
operation_plan

,power_sell,power_buy,energy_cost,grid_energy_cost,battery_state
timestamps,,,,,
2019-05-29 14:00:00,0.000000,0.0,0.000000,0.000000,298.261657
2019-05-29 15:00:00,10.894504,0.0,-4.404648,0.163418,416.422644
2019-05-29 16:00:00,30.022187,0.0,-13.527998,0.450333,416.422644
2019-05-29 17:00:00,0.000000,0.0,0.000000,0.000000,314.418654
2019-05-29 18:00:00,0.000000,0.0,0.000000,0.000000,145.703452
...,...,...,...,...,...
2020-03-13 09:00:00,314.276904,0.0,-14.016750,4.714154,0.000000
2020-03-13 10:00:00,206.493110,0.0,-9.230242,3.097397,0.000000
2020-03-13 11:00:00,255.452400,0.0,-10.243641,3.831786,0.000000


In [22]:
grid_power_contract

479.8

In [23]:
peak_contract_difference

0.0

In [24]:
grid_power_cost

53256.6

In [25]:
total_cost

210218.0